In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.tree import DecisionTreeClassifier
from collections import Counter, defaultdict
from sklearn.naive_bayes import BernoulliNB
from IPython.display import display, Markdown
import json

In [3]:
import sys
import os
tornado_path = os.path.abspath('../../tornado')
sys.path.insert(0, tornado_path)
from drift_detection.__init__ import *

In [119]:
# I am here assuming that there is no 'date' column

# All of these scenario functions return:
#   1. a copy of the data with the scenario applied to it
#   2. the streams which should experience change

# All scenarios should experience a change in the error rate

def shuffle_features(data, classes, shuffle_n=5):
    # pick shuffle_n features columns and shuffle them
    ret = data.copy()
    features = set(data.columns) - set(['label'])
    to_shuffle = random.sample(features, shuffle_n)
    for i in range(shuffle_n):
        ret[to_shuffle[i]] = ret[to_shuffle[(i+1)%shuffle_n]]
    return ret, to_shuffle + [f'label={i}' for i in classes]
    
def not_feature(data, classes):
    # randomly choose a feature x and replace values with not x
    ret = data.copy()
    features = set(data.columns) - set(['label'])
    invert_col = random.choice(list(features))
    ret[invert_col] = [ not i for i in ret[invert_col] ]
    return ret, [invert_col] + [f'label={i}' for i in classes]

def swap_classes(data, classes):
    # pick two classes and replace half of instances of class 1 with instances of class 2
    ret = data.copy()
    active_classes = list(set(data['label']))
    incr_class, decr_class = random.sample(active_classes, 2)
    replace_with = data[ data['label'] == incr_class ]
    to_replace = np.where(data['label'] == decr_class )[0] # output is wrapped in a tuple for some reason
    to_replace = list(to_replace)
    replace_n = len(to_replace) // 2
    to_replace = random.sample(to_replace, replace_n)
    for replace_i in to_replace:
        replacer = replace_with.sample(1).values
        for j in range(len(ret.columns)):
            ret.iloc[replace_i, j] = replacer[0][j]
#         ret.iloc[replace_i, :] = 
    features = set(data.columns) - set(classes)
    return ret, [f'label={i}' for i in [decr_class, incr_class] ] + list(features) # class rates should change plus all feature rates

def new_concept(data, classes):
    # drop classes columns
    no_classes = data.drop(columns=['label'])
    # train new concept
    dtree = DecisionTreeClassifier()
    n_subsamples = 10
    dtree.fit(no_classes.sample(n_subsamples), random.choices(list(data['label']), k=n_subsamples))
    # predict labels
    labels = dtree.predict(no_classes)
    # reset classes
    ret = data.copy()
    ret['label'] = labels
    return ret, [f'label={i}' for i in classes] + ['err'] # dtree

scenarios = 'shuffle_features not_feature swap_classes new_concept'.split()

def add_made_up_labels(no_classes, classes):
    # NB: no_classes is NOT number of classes. It's a df without any classes/labels
    # train new concept
    dtree = DecisionTreeClassifier()
    n_subsamples = 10
    dtree.fit(no_classes.sample(n_subsamples), random.choices(classes, k=n_subsamples))
    # predict labels
    labels = dtree.predict(no_classes)
    # reset classes
    ret = no_classes.copy()
    ret['label'] = labels
    return ret

def detect_stream_drift(stream, detector):
    # NB stream should be a list
    for i in range(len(stream)):
        _, drift_status = detector.run(stream[i])
        if drift_status == True:
            return i
    return False

def detect_drifting_streams(data, detector_algorithm, delta):
    # returns dictionary mapping column name -> point of drift detection
    # for all the columns in which drift was detected
    ret = {}
    for col in data.columns:
        detector = detector_algorithm()#delta=delta)
        stream = list(data[col])
        detection = detect_stream_drift(stream, detector)
        ret[col] = detection
#         if detection:
#             ret[col] = detection
    return ret

DRIFT_POINT = 500

class ResultCounter(dict):
    def __init__(self):
        super(ResultCounter, self).__init__()
        start = 0
        self['tp'] = start
        self['fp'] = start
        self['tn'] = start
        self['fn'] = start

def evaluate_detector(features, labels, err, classes, detector_algorithm, delta, drifting_streams):
    # drifting streams is the correct labels of which are drifting
    
    detections = {}
    
    # err
    err_df = pd.DataFrame({'err': err})
    detections['err'] = detect_drifting_streams(err_df, detector_algorithm, delta)
    
    # labels
    labels = list(labels)
    label_df = pd.DataFrame({f'label={class_}': [False for i in range(len(labels))] for class_ in classes})
    for i in range(len(labels)):
        label_df.loc[i, f'label={labels[i]}'] = True
    detections['label'] = detect_drifting_streams(label_df, detector_algorithm, delta)
    
    # features
    detections['feature'] = detect_drifting_streams(features, detector_algorithm, delta)
    
    metrics = defaultdict(ResultCounter)
    for thing in ['err', 'label', 'feature']:
        det = detections[thing]
        metrics[thing]['delay'] = []
        for col in det.keys():
            col_det = det[col]
            if col_det == False:
                if col in drifting_streams:
                    metrics[thing]['fn'] += 1
                else:
                    metrics[thing]['tn'] += 1
            elif col_det and col_det < DRIFT_POINT:
                metrics[thing]['fp'] += 1
            else:
                if col in drifting_streams:
                    metrics[thing]['tp'] += 1
                    metrics[thing]['delay'].append(col_det)
                else:
                    metrics[thing]['fp'] += 1
                    
    # Convert 'delay' from list of delays to mean delay
    for thing in metrics.keys():
        delays = metrics[thing]['delay']
        if len(delays) > 0:
            metrics[thing]['delay'] = np.mean(delays) - DRIFT_POINT
        else:
            metrics[thing]['delay'] = 'na'
                    
    # Don't bother recording non-detections
    detections = { thing: { col: i for col, i in thing_.items() if i } 
                  for thing, thing_ in detections.items() }
    return {'metrics': dict(metrics), 'detections': detections}
    
def generate_dataset(train, test, scenario):
    
    # train model
    model = learner()
    model.fit(train.drop(columns=['label']), train['label'])
    
    # Step 1: new instane arrives
    split_at = DRIFT_POINT
    test1 = test.iloc[:split_at, :]
    test2 = test.iloc[split_at:, :]
    scenario_f = eval(scenario)
    test2, changing_streams = scenario_f(test2, classes)
    test = pd.concat([test1, test2])
    changing_streams += ['err']
    
    # Step 2: model makes predictions
    y_hat = model.predict(test.drop(columns='label'))
    
    # Step 3: true label arrives
    y = test['label']
    test.drop(columns='label', inplace=True)
    errs = [ i!=j for i,j in zip(y, y_hat) ]
    
    dirname = os.path.abspath('./scenarios/' + scenario)
    
    if not os.path.exists(dirname):
        os.makedirs(dirname)
    
    test.to_csv(dirname + '/test.csv', index=False)
    with open(dirname + '/y.json', 'w') as f:
        f.write(json.dumps(list(y)))
    with open(dirname + '/errs.json', 'w') as f:
        f.write(json.dumps([int(i) for i in errs]))
    with open(dirname + '/changing_streams.json', 'w') as f:
        f.write(json.dumps(changing_streams))
        

def run_experiment(train, test, classes, learner, scenario, detector_algorithm, delta):
    
    dirname = os.path.abspath('./scenarios/' + scenario)
    
    test = pd.read_csv(dirname + '/test.csv')
    with open(dirname + '/y.json', 'r') as f:
        y = json.loads(f.read())
    with open(dirname + '/errs.json', 'r') as f:
        errs = json.loads(f.read())
    with open(dirname + '/changing_streams.json', 'r') as f:
        changing_streams = json.loads(f.read())
    
    # run drift detector
    # no_comparisons = len(classes) + len(train.columns) # minus one for 'label' and plus one for 'err'
    # delta = delta / no_comparisons # bonferonni correction
    result = evaluate_detector(test, y, errs, classes, detector_algorithm, delta, changing_streams)
    
    return result

In [120]:
# generate datasets
data = pd.read_csv('synthetic.csv').drop(columns='date')
classes = [1,2,3,4,5]
data = add_made_up_labels(data, classes)
test = data.loc[500:1500, :]
train = data.loc[:500, :]

for scenario in scenarios:
    generate_dataset(train, test, scenario)

(500, 114)
(501, 114)
(500, 114)
(501, 114)
(500, 114)
(501, 114)
(500, 114)
(501, 114)


### Scenarios:
 * Feature importance drift?
 * Suppose features are drifting
   * All features will drift with probability $p$
   * Express $p$ in terms of $E[N$ drifting features$]$ or $\Pr($given feature drifting$)$ values.
   * After several runs calculate the precision and recall of detecting shifted features
   * Assumption: feature drifts are uncorrelated (so we can use the Szigly drift detector)
   * But this is going to entirely depend on the characteristics of the particular feature stream.
 * Is it possible to distinguish between different causal paths:
   * Class -> features
   * Features -> class
   * Features -> err
   * Err -> class
 * Irrelevant feature shifts

# Scenarios
 * move from higher to lower priorities
 * increase/decrease noise
 * 

In [121]:
delta = 0.5 # currently this doesn't do anything
learner = BernoulliNB

detectors = ['ADWINChangeDetector', 'CUSUM', 'DDM', 'EDDM', 'FHDDM', 'FHDDMS', 'HDDM_A_test', 
              'HDDM_W_test', 'RDDM', 'PH', 'SeqDrift2ChangeDetector' ]

results = { detector:
    { scenario: run_experiment(train, test, classes, learner, scenario, eval(detector), delta)
     for scenario in scenarios }
    for detector in detectors }

In [123]:
for thing in ['err', 'feature', 'label']:
    display(Markdown('# '+thing))
    results_df = pd.DataFrame(index=detectors, columns=scenarios)
    for detector in detectors:
        for scenario in scenarios:
            metrics = results[detector][scenario]['metrics'][thing]
            tp = metrics['tp']
            fp = metrics['fp']
            fn = metrics['fn']
            try:
                prec = tp / (tp + fp)
            except ZeroDivisionError:
                prec = 0.
            try:
                rec = tp / (tp + fn)
            except ZeroDivisionError:
                rec = 0.
            results_df.loc[detector, scenario] = f'{rec:.2f} ({prec:.2f})'
    display(results_df)

# err

,shuffle_features,not_feature,swap_classes,new_concept
ADWINChangeDetector,0.00 (0.00),0.00 (0.00),0.00 (0.00),1.00 (1.00)
CUSUM,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
DDM,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
EDDM,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
FHDDM,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
FHDDMS,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
HDDM_A_test,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
HDDM_W_test,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
RDDM,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
PH,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)


# feature

,shuffle_features,not_feature,swap_classes,new_concept
ADWINChangeDetector,0.60 (1.00),1.00 (1.00),0.00 (0.00),0.00 (0.00)
CUSUM,0.40 (1.00),1.00 (1.00),0.00 (0.00),0.00 (0.00)
DDM,0.50 (0.25),1.00 (0.14),0.00 (0.00),0.00 (0.00)
EDDM,0.67 (0.02),0.00 (0.00),0.11 (0.03),0.00 (0.00)
FHDDM,0.40 (0.50),1.00 (0.33),0.01 (1.00),0.00 (0.00)
FHDDMS,0.40 (0.33),1.00 (0.20),0.04 (1.00),0.00 (0.00)
HDDM_A_test,0.40 (1.00),1.00 (1.00),0.00 (0.00),0.00 (0.00)
HDDM_W_test,0.40 (0.13),1.00 (0.07),0.08 (0.82),0.00 (0.00)
RDDM,0.50 (0.04),1.00 (0.04),0.10 (0.42),0.00 (0.00)
PH,0.40 (1.00),1.00 (1.00),0.00 (0.00),0.00 (0.00)


# label

,shuffle_features,not_feature,swap_classes,new_concept
ADWINChangeDetector,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.20 (1.00)
CUSUM,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
DDM,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.20 (1.00)
EDDM,0.00 (0.00),0.00 (0.00),0.00 (0.00),1.00 (0.20)
FHDDM,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.20 (1.00)
FHDDMS,0.20 (1.00),0.20 (1.00),0.00 (0.00),0.20 (1.00)
HDDM_A_test,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)
HDDM_W_test,0.20 (1.00),0.20 (1.00),0.00 (0.00),0.20 (1.00)
RDDM,0.25 (0.50),0.25 (0.50),0.50 (0.33),0.50 (0.67)
PH,0.00 (0.00),0.00 (0.00),0.00 (0.00),0.00 (0.00)


In [ ]:
# I am here assuming that there is no 'date' column

def shuffle_features(data, classes, shuffle_n=5):
    # pick shuffle_n features columns and shuffle them
    ret = data.copy()
    features = set(data.columns) - set(classes)
    to_shuffle = random.sample(features, shuffle_n)
    for i in range(shuffle_n):
        ret[to_shuffle[i]] = re[to_shuffle[(i+1)%shuffle_n]]
    return ret, to_shuffle
    
def not_feature(data, classes):
    # randomly choose a feature x and replace values with not x
    ret = data.copy()
    features = set(data.columns) - set(classes)
    invert_col = random.choice(features)
    ret[invert_col] = [ not i for i in ret[invert_col] ]
    return ret, invert_col

def incr_class_rate(data, classes):
    # pick a class and double its rate
    # TODO: currently actually converts half of non-x's to x's
    ret = data.copy()
    incr_class = random.choice(classes)
    replacements = data[ data[incr_class] == True ]
    for i in range(len(data)):
        if not ret.loc[i, incr_class]:
            if random.random() < 0.5:
                # with 50% probability replace this class with incr_class
                ret.loc[i, :] = replacements.sample(1).values
    return ret, incr_class
                
def decr_class_rate(data, classes):
    # pick a class and half its rate
    # TODO
    ret = data.copy()
    decr_class = random.choice(classes)
    replacements = data[ data[decr_class] == False ]
    for i in range(len(data)):
        if ret.loc[i, decr_class]:
            if random.random() < 0.5:
                # with 50% probability replace this class with incr_class
                ret.loc[i, :] = replacements.sample(1).values
    return ret, decr_class

def new_concept(data, classes):
    # drop classes columns
    no_classes = data[ [ col for col in data.columns if col not in classes ] ]
    # train new concept
    dtree = DecisionTreeClassifier()
    n_subsamples = 10
    dtree.fit(no_classes.sample(n_subsamples), random.choices(range(len(classes)), k=n_subsamples))
    # predict labels
    labels = dtree.predict(no_classes)
    # reset classes
    ret = data.copy()
    for class_ in classes:
        ret[class_] = [ False for i in range(len(data[class_])) ]
    # re-assign classes
    for i in range(len(ret)):
        ret.loc[i, classes[labels[i]]] = True
    return ret, dtree

scenarios = 'shuffle_features not_feature incr_class_rate decr_class_rate new_concept'.split()

def run_experiment(train, test, classes, learner, scenario, detector):
    # train and test should initially have a single 'label' column
    model = learner()
    model.fit(train[[ col for col in train.columns if col != 'label' ]], train['label'])
    
    # split 
                
        
                

In [112]:
def print_dtree(dtree):
    dot_data = StringIO()
    export_graphviz(dtree, out_file=dot_data,  
                    filled=True, rounded=True,
                    special_characters=True)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())  
    
    display(Markdown(f'### Concept {i}'))
    display((Image(graph.create_png())))

    split_features = [ i for i in dtree.tree_.feature if i >= 0 ]
    print(f'Split features: {list(dataset.columns[split_features])}')